In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
def jobplanet_job(result):
    keywords = ['백엔드 개발', '프론트엔드 개발', '클라우드 개발', '게임개발', '네트워크/보안/운영', '소프트웨어 개발', '웹개발']

    wd = webdriver.Chrome()

    for keyword in keywords:
        url = "https://www.jobplanet.co.kr/job"

        wd.get(url)
        time.sleep(1)

        try:
            # 1단계: "백엔드 개발" 같은 직무 필터 버튼 클릭 (왼쪽 위)
            job_filter_button = wd.find_element(By.XPATH, "//a[contains(@class, 'FilterSelect') or contains(text(), '백엔드 개발') or contains(text(), '직무')]")
            job_filter_button.click()
            time.sleep(2)
            print(f'{keyword}: 필터 메뉴 열기 성공')

            # 2단계: 왼쪽 사이드바에서 "개발" 카테고리 클릭
            coding_category = wd.find_element(By.XPATH, "//button[text()='개발']")
            coding_category.click()
            time.sleep(1)
            print(f'{keyword}: 개발 카테고리 클릭 성공')

            # 3단계: 원하는 직무 체크박스 클릭
            job_checkbox = wd.find_element(By.XPATH, f"//label[contains(text(), '{keyword}')]")
            job_checkbox.click()
            time.sleep(1)
            print(f'{keyword}: 직무 선택 성공')

            # 4단계: 적용 버튼 클릭
            apply_button = wd.find_element(By.XPATH, "//button[contains(text(), '적용')]")
            apply_button.click()
            time.sleep(3)
            print(f'{keyword}: 적용 버튼 클릭 성공')
            """
            carrer_filter = wd.find_element(By.XPATH, "//a[contains(text(), '직종')]")
            carrer_filter.click()
            time.sleep(2)

            coding_button = wd.find_element(By.XPATH, "//button[contains(text(), '개발')]")
            coding_button.click()
            time.sleep(1)

            job_button = wd.find_element(By.XPATH, f"//a[contains(text(), '{keyword}')]")
            job_button.click()
            time.sleep(2)
            
            """
            for i in range(15):
                wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
            
            html = wd.page_source
            soup = BeautifulSoup(html, 'html.parser')

            jop_stack_list = soup.find_all('span', class_='mt-[7px] inline-block w-full truncate text-small1 text-gray-500')

            jop_stack_result = []

            for jop_stack in jop_stack_list:
                text = jop_stack.text

                if ',' in text:
                    parts = text.split(',')

                    if '년' in parts[0]:
                        stacks = parts[1:]
                    else:
                        stacks = parts

                    for s in stacks:
                        if s.strip():
                            jop_stack_result.append(s.strip())
            
            jop_stack_count = {}
            for count in jop_stack_result:
                if count in jop_stack_count:
                    jop_stack_count[count] = jop_stack_count[count] + 1
                else:
                    jop_stack_count[count] = 1

            for tech, count_num, in jop_stack_count.items():
                result.append([keyword, tech, count_num])

        except:
            continue
    
    return

In [10]:
def visulaize_stack(df):
    plt.rcParams['font.family'] = 'Malgun Gothic'
    plt.rcParams['axes.unicode_minus'] = False

    df['빈도수'] = pd.to_numeric(df['빈도수'])

    ranking = df.groupby('직무별 스택')['빈도수'].sum().nlargest(20).index
    df_ranking = df[df['직무별 스택'].isin(ranking)]

    pivot = df_ranking.pivot_table(values='빈도수', index='직무', columns='직무별 스택', fill_value=0)

    plt.figure(figsize=(19, 9))
    sns.heatmap(pivot, annot=True, fmt='.0f', cmap='YlOrRd', linewidths=0.5)

    plt.title('직무별 기술 스택')
    plt.xlabel('기술 스택')
    plt.ylabel('직무')
    plt.xticks(rotation=50)
    plt.tight_layout()

    plt.show()

In [11]:
def main():
    result = []

    jobplanet_job(result)

    jobplanet_df = pd.DataFrame(result, columns=['직무', '직무별 스택', '빈도수'])

    jobplanet_df.to_csv('C:/Users/wjdah/vscode-workspace/CareerRoute/assets/jobplanet/jobplanet_stack.csv', encoding='utf-8-sig', index='False')

    visulaize_stack(jobplanet_df)

In [12]:
if __name__ == '__main__':
    main()

ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 1900x900 with 0 Axes>